In [15]:
from ipywidgets import widgets
from IPython.display import display

slider = widgets.IntSlider()
display(slider)

IntSlider(value=0)

In [17]:
!pip install langchain
%pip install -qU pypdf

In [18]:
!pip install langchain_community

In [19]:
# 1. Import libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os

In [20]:
# 2. Load the PDF document
pdf_path = "Exxonmobil_Report.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()  # Extracts text from each page as separate Document objects

In [21]:
# 3. Split the PDF into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)  # Helps the retriever by chunking large documents

In [22]:
from langchain_community.vectorstores import FAISS
!pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.3 MB/s eta 0:00:00


In [23]:
# 4. Embed the text chunks using a sentence-transformer
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(docs, embedding_model)  # Creates vector index using FAISS


/tmp/ipython-input-23-4050901680.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
from huggingface_hub import notebook_login # Import notebook_login

# Log in to Hugging Face - this will prompt you to enter your token
# Ensure you have accepted the model's terms and conditions on the HF website
try:
    notebook_login()
except Exception as e:
    print(f"Hugging Face login failed: {e}")
    print("Please make sure you have accepted the terms for the model on the Hugging Face website and have a valid token.")

In [26]:
# 5. Load the open-source LLM (e.g., Mistral 7B)
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [27]:
# 6. Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1
)

Device set to use cuda:0


In [28]:
# 7. Wrap the pipeline for LangChain
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipython-input-28-3267213372.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [35]:
# 8. Add memory to maintain chat history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer" # Explicitly set the output key to save the 'answer'
)

In [36]:
# 9. Create Conversational Retrieval Chain (RAG pipeline)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(),
    memory=memory,
    return_source_documents=True
)

In [37]:
# 10. Run a simple loop to chat with the PDF
print("📘 Ask questions about the PDF (type 'exit' to quit):")
chat_history = []

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit"]:
        break

    result = qa_chain({"question": query})
    print("\n🤖 Bot:", result["answer"])

📘 Ask questions about the PDF (type 'exit' to quit):

You: Summarize the document and its findings.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

performings ucho ther procedures as we considered necessary in thec ircumstances.W eb elieve that our audits provide ar easonable
basisf or our opinions.
As describedi nM anagement’sR eporto nI nternalC ontrolo verF inancial Reporting, management hase xcludedP ioneer Natural
ResourcesC ompany ("Pioneer") fromi ts assessmento fi nternalc ontrolo verf inancial reportinga so fD ecember 31, 2024 because it

requirements.
NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
97

returns through disciplined  
capital allocation
See supplemental information for footnotes, definitions, and cautionary statement beginning 
on page 138 in this report.
143

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
115

Question: Summarize the document and its findings.
Helpful Answer: The document is Management's Report on Internal Control over Financial

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

consolidated financials tatements, whetherd ue to erroro rf raud, andp erformingp rocedures that respond to thoser isks.S uch
procedures includede xamining,o na test basis, evidence regardingt he amountsa nd disclosuresi nt he consolidated financial
statements.O ur audits also includede valuatingt he accountingp rinciplesu seda nd significante stimatesm adeb ym anagement, as well

performings ucho ther procedures as we considered necessary in thec ircumstances.W eb elieve that our audits provide ar easonable
basisf or our opinions.
As describedi nM anagement’sR eporto nI nternalC ontrolo verF inancial Reporting, management hase xcludedP ioneer Natural
ResourcesC ompany ("Pioneer") fromi ts assessmento fi nternalc ontrolo verf inancial reportinga so fD ecember 31, 2024 because it

requirements.
NOTES TO CONSOLIDATED F

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

consolidated financials tatements, whetherd ue to erroro rf raud, andp erformingp rocedures that respond to thoser isks.S uch
procedures includede xamining,o na test basis, evidence regardingt he amountsa nd disclosuresi nt he consolidated financial
statements.O ur audits also includede valuatingt he accountingp rinciplesu seda nd significante stimatesm adeb ym anagement, as well

performings ucho ther procedures as we considered necessary in thec ircumstances.W eb elieve that our audits provide ar easonable
basisf or our opinions.
As describedi nM anagement’sR eporto nI nternalC ontrolo verF inancial Reporting, management hase xcludedP ioneer Natural
ResourcesC ompany ("Pioneer") fromi ts assessmento fi nternalc ontrolo verf inancial reportinga so fD ecember 31, 2024 because it

requirements.
NOTES TO CONSOLIDATED F